# Benchmark:
- Julia version: 1.2.0
- Author: laurent.berge
- Date: 2019-11-13
- ~: Replication in Julia

- Julia version: 1.7.1
- Author 2: Miguel Portela
- Date: 2022-06-20

In [4]:
VERSION

v"1.7.1"

In [5]:
cd("/Users/miguelportela/Documents/GitHub/HDFE")
homedir()
pwd()

"/Users/miguelportela/Documents/GitHub/fixed_effects_estimation/PRJS"

In [6]:
import Pkg

In [7]:
#Pkg.add("RDatasets")
#Pkg.add("ReadStat")
#Pkg.add("StatFiles")
#Pkg.add("CSV")

   Resolving package versions...
  No Changes to `~/opt/anaconda3/envs/mypython310/share/julia/environments/mypython310/Project.toml`
  No Changes to `~/opt/anaconda3/envs/mypython310/share/julia/environments/mypython310/Manifest.toml`


In [8]:
println("Loading packages...")
using RData, DelimitedFiles, FixedEffectModels, DataFrames, CSV, RDatasets, ReadStat, StatFiles
println("done.")

Loading packages...
done.


In [9]:
#nlswork = read_dta("nlswork_regression.dta")
nlswork = CSV.read("nlswork_regression.csv", DataFrame)
#nlswork = DataFrame(load("nlswork_regression.dta"))

,idcode,year,birth_yr,age,race,msp,nev_mar,grade,collgrad,not_smsa
,Int64,Int64,Int64,Int64?,String7,Int64?,Int64?,Int64?,Int64,Int64?
1,1,70,51,18,Black,0,1,12,0,0
2,1,71,51,19,Black,1,0,12,0,0
3,1,72,51,20,Black,1,0,12,0,0
4,1,73,51,21,Black,1,0,12,0,0
5,1,75,51,23,Black,1,0,12,0,0
6,1,77,51,25,Black,0,0,12,0,0
7,1,78,51,26,Black,0,0,12,0,0
8,1,80,51,28,Black,0,0,12,0,0
9,1,83,51,31,Black,0,0,12,0,0


In [14]:
timings_base = @elapsed reg(nlswork, @formula(ln_wage ~ ttl_exp + union + not_smsa + nev_mar + fe(idcode) + fe(year)))
#reg(nlswork, @formula(ln_wage ~ ttl_exp + union + not_smsa + nev_mar + fe(idcode) + fe(year)))

0.307732959

In [19]:
@elapsed HDFE4 = reg(nlswork, @formula(ln_wage ~ ttl_exp + union + not_smsa + nev_mar + fe(idcode) + fe(year)))

0.317983375

In [20]:
HDFE4

                           Fixed Effect Model                           
Number of obs:                18558   Degrees of freedom:            3500
R2:                           0.756   R2 Adjusted:                  0.700
F-Stat:                     266.997   p-value:                      0.000
R2 within:                    0.066   Iterations:                      11
ln_wage  |   Estimate  Std.Error  t value Pr(>|t|)  Lower 95%   Upper 95%
-------------------------------------------------------------------------
ttl_exp  |  0.0436694 0.00159399  27.3963    0.000   0.040545   0.0467938
union    |   0.100953 0.00691049  14.6086    0.000  0.0874072    0.114498
not_smsa | -0.0942986  0.0123715 -7.62227    0.000  -0.118548   -0.070049
nev_mar  | -0.0209187  0.0102182 -2.04721    0.041 -0.0409476 -0.00088992


In [14]:
df = dataset("plm", "Cigar")
reg(df, @formula(Sales ~ NDI + fe(State) + fe(Year)), Vcov.cluster(:State), weights = :Pop)

                           Fixed Effect Model                           
Number of obs:                 1380  Degrees of freedom:              32
R2:                           0.803  R2 Adjusted:                  0.798
F-Stat:                     13.3382  p-value:                      0.001
R2 within:                    0.139  Iterations:                       5
Sales |    Estimate  Std.Error  t value Pr(>|t|)   Lower 95%   Upper 95%
------------------------------------------------------------------------
NDI   | -0.00526264 0.00144097 -3.65216    0.000 -0.00808942 -0.00243586


In [15]:
println("Loading data...")
base_all = load("DATA/base_all_simulations.RData", convert=true);
base_all = base_all["base_all"];
println(".")
base_10M = CSV.read("DATA/base_10M.csv", DataFrame)
println("done.")

Loading data...
.
done.


# warming up

In [16]:
println("warming up...")
base = base_all[1]
test = @elapsed reg(base, @formula(ln_y~X1 + fe(dum_1)))
println("done.")

warming up...
done.


# 1 FE

# We run all the models

In [6]:
println("One FE.")
timings = Vector{Float64}(undef, 50)
for i=1:40
    println("i = ", i)
    base = base_all[i]
    print(size(base))
    timings[i] = @elapsed reg(base, @formula(ln_y~X1 + fe(dum_1)))
end

One FE.
i = 1
(1000, 6)i = 2
(10000, 6)i = 3
(100000, 6)i = 4
(1000000, 6)i = 5
(1000, 6)i = 6
(10000, 6)i = 7
(100000, 6)i = 8
(1000000, 6)i = 9
(1000, 6)i = 10
(10000, 6)i = 11
(100000, 6)i = 12
(1000000, 6)i = 13
(1000, 6)i = 14
(10000, 6)i = 15
(100000, 6)i = 16
(1000000, 6)i = 17
(1000, 6)i = 18
(10000, 6)i = 19
(100000, 6)i = 20
(1000000, 6)i = 21
(1000, 6)i = 22
(10000, 6)i = 23
(100000, 6)i = 24
(1000000, 6)i = 25
(1000, 6)i = 26
(10000, 6)i = 27
(100000, 6)i = 28
(1000000, 6)i = 29
(1000, 6)i = 30
(10000, 6)i = 31
(100000, 6)i = 32
(1000000, 6)i = 33
(1000, 6)i = 34
(10000, 6)i = 35
(100000, 6)i = 36
(1000000, 6)i = 37
(1000, 6)i = 38
(10000, 6)i = 39
(100000, 6)i = 40
(1000000, 6)

# The 10M DB

In [7]:
for i=1:10
    println("i = ", i + 40)
    timings[i + 40] = @elapsed reg(base_10M, @formula(ln_y~X1 + fe(dum_1)))
end

i = 41
i = 42
i = 43
i = 44
i = 45
i = 46
i = 47
i = 48
i = 49
i = 50


# saving

In [8]:
open("DATA/julia_bench_1FE.txt", "w") do io
           writedlm(io, timings)
       end

# 2 FE

In [9]:
# We run all the models
println("Two FEs.")
timings = Vector{Float64}(undef, 50)
for i=1:40
    println("i = ", i)
    base = base_all[i]
    timings[i] = @elapsed reg(base, @formula(ln_y~X1 + fe(dum_1) + fe(dum_2)))
end

# The 10M DB
for i=1:10
    println("i = ", i + 40)
    timings[i + 40] = @elapsed reg(base_10M, @formula(ln_y~X1 + fe(dum_1) + fe(dum_2)))
end

# saving
open("DATA/julia_bench_2FE.txt", "w") do io
           writedlm(io, timings)
       end

Two FEs.
i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
i = 10
i = 11
i = 12
i = 13
i = 14
i = 15
i = 16
i = 17
i = 18
i = 19
i = 20
i = 21
i = 22
i = 23
i = 24
i = 25
i = 26
i = 27
i = 28
i = 29
i = 30
i = 31
i = 32
i = 33
i = 34
i = 35
i = 36
i = 37
i = 38
i = 39
i = 40
i = 41
i = 42ean Variables: [================================>]  2/2
i = 43ean Variables: [================================>]  2/2
i = 44ean Variables: [================================>]  2/2
i = 45ean Variables: [================================>]  2/2
i = 46ean Variables: [================================>]  2/2
i = 47ean Variables: [================================>]  2/2
i = 48ean Variables: [================================>]  2/2
i = 49ean Variables: [================================>]  2/2
i = 50ean Variables: [================================>]  2/2
Demean Variables: [================================>]  2/2

# 3 FE

In [10]:
# We run all the models
println("Three FEs.")
timings = Vector{Float64}(undef, 50)
for i=1:40
    println("i = ", i)
    base = base_all[i]
    timings[i] = @elapsed reg(base, @formula(ln_y~X1 + fe(dum_1) + fe(dum_2) + fe(dum_3)))
end

# The 10M DB
for i=1:10
    println("i = ", i + 40)
    timings[i + 40] = @elapsed reg(base_10M, @formula(ln_y~X1 + fe(dum_1) + fe(dum_2) + fe(dum_3)))
end

# saving
open("DATA/julia_bench_3FE.txt", "w") do io
           writedlm(io, timings)
       end

Three FEs.
i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
i = 10
i = 11
i = 12
i = 13
i = 14
i = 15
i = 16
i = 17
i = 18
i = 19
i = 20
i = 21
i = 22
i = 23
i = 24
i = 25
i = 26
i = 27
i = 28
i = 29
i = 30
i = 31
i = 32
i = 33
i = 34
i = 35
i = 36
i = 37
i = 38
i = 39
i = 40
i = 41
i = 42ean Variables: [================================>]  2/2
i = 43ean Variables: [================================>]  2/2
i = 44ean Variables: [================================>]  2/2
i = 45ean Variables: [================================>]  2/2
i = 46ean Variables: [================================>]  2/2
i = 47ean Variables: [================================>]  2/2
i = 48ean Variables: [================================>]  2/2
i = 49ean Variables: [================================>]  2/2
i = 50ean Variables: [================================>]  2/2
Demean Variables: [================================>]  2/2

#
# The "difficult" data
#


In [11]:
println("Loading difficult data...")
base_diff = load("DATA/base_all_diff.RData", convert=true);
base_diff = base_diff["base_all_diff"];
println("done.")

# warming up
println("warming up...")
base = base_diff[1]
test = @elapsed reg(base, @formula(y ~ x1 + x2 + fe(id_indiv)))
println("done.")

timings = Vector{Float64}(undef, 120)

global index = 1
for size=1:4
    println("size = ", size)
    base = base_diff[size]
    for g=1:3
        println("g = ", g)
        for r=1:10
            if g == 1
                global timings[index] = @elapsed reg(base, @formula(y ~ x1 + x2 + fe(id_indiv)))
            elseif g == 2
                global timings[index] = @elapsed reg(base, @formula(y ~ x1 + x2 + fe(id_indiv) + fe(id_firm)))
            else
                global timings[index] = @elapsed reg(base, @formula(y ~ x1 + x2 + fe(id_indiv) + fe(id_firm) + fe(id_year)))
            end
            global index = index + 1
        end
    end
end

# saving
open("DATA/julia_bench_diff.txt", "w") do io
           writedlm(io, timings)
       end

Loading difficult data...
done.
warming up...
done.
size = 1
g = 1
g = 2
g = 3
size = 2
g = 1
g = 2
g = 3mean Variables: [================================>]  3/3    ]  1/3
size = 3n Variables: [================================>]  3/3    ]  1/3
g = 1
g = 2
g = 3mean Variables: [================================>]  3/3    ]  1/3
    Demean Variables: [======================                     ]  1/3